### <p style="text-align: right;"> &#9989; Dylan Miskowski</p>

# CMSE 202 Homework 03






### Assignment instructions

Work through the following assignment, making sure to follow all of the directions and answer all of the questions.

There are **25 points** possible on this assignment. Point values for each part are included in the section headers.

This assignment is due at 11:59 pm on **Friday October 23rd**. It should be uploaded into the "Homework Assignments" submission folder for Homework 3 in your D2L webpage. Submission instructions can be found at the end of the notebook.

**Hint**: It is possible you are asked to do something you are not familiar with. That's why you have internet access. Do some smart searches and see what you can find! 


### Our imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix


## Part 1: Setting up a repository for tracking changes (3 points)

For this assignment, you're going to add it to the CMSE202 repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to:

* Navigate to your `/CMSE202/repos` repository and create a new directory called `hw-03`.
* Move this notebook into that new directory in your repository, then add it and commit it to your repository.
 * Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

Important: Make sure you've added your TA as a collaborators\ to your respository with "Read" access so that we can see your assignment.

* Section 001:  tuethan
* Section 002:  Luis-Polanco
* Section 003:  DavidRimel

Also important: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the noteobok, none of your changes will be tracked.


If everything went as intended, the file should now show up on your GitHub account CMSE202 repository under the `hw-03` directory that you just created. Periodically, you'll be asked to commit your changes to the repository and push them to the remote GitHub location. Of course, you can always commit your changes more often than that, if you wish. It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

## Part 2: Load, prepare and plot the data (5 points)

In this homework we will be working with the yeast dataset and building logistic regression and k-nearest neighbors classifier class. The data file is *yeast.data* and its description is in *yeast.names*. Read the description and get a sense of the meaning of the dataset. In this part, we will load and clean up the data.

**Question 2.1 (1 point)** Load the *yeast.data* as a pandas dataframe and give appropriate names to the columns. Then drop the columns **sequence name**, **pox** and **vac**. What's the size of this dataset now?

In [ ]:
#load yeast.data as df, give names to columns
names=['Sequence name', 'mcg','gvh','alm','mit','erl','pox','vac','nuc','class']

yeast=pd.read_fwf('yeast.data', names=names)

#drop sequence name, pox and vac
yeast=yeast.drop(['Sequence name','pox','vac'], axis=1)
#size of df?
yeast

#it is now 1484 rows x 7 columns


**Question 2.2 (1 point)** Find the number of unqiue entries in the class label column

In [ ]:
#10 unique entries (from yeast.names)

**Question 2.3 (1 point)** We are only interested in data with label **CYT (cytosolic or cytoskeletal)** and **MIT (mitochondrial)**. Make a new dataframe containing
data with only these two types of labels, and redefine label **CYT** into **0**, and **MIT** into **1**. What's the size of the dataset now?

In [ ]:
#only want class CYT and MIT
#crit1=yeast['class']=='CYT'

yeast2=yeast[yeast['class'].isin(['CYT','MIT'])] #OMG USE THIS FOR SELECTING 2 FROM COLUMNS FOR NOW ON!!!


#redefine CYT to 0 and MIT to 1

yeast2=yeast2.replace('CYT',0)
yeast2=yeast2.replace('MIT',1)

yeast2

**Question 2.4 (2 points)** Make a scatter plot including every sample in the dataset with: the mcg feature on the x-axis, the gvh feature on the y-axis, and different colors for each class label. Make your observation. Are the two classes distinguishable using only those two features?

In [ ]:
### Put your code here ###
#mcg on x, gvh on y and different colors for each label

yeast2.plot(x='mcg', y='gvh',kind='scatter', c='class', cmap='Accent', colorbar=False,legend=True)


#Q: are the classes distinguishable using only those 2 features?
#A: no I would say we cannot easily sep them with only those 2 features. Everything is very intermingled. 

# Logistic Regression

In the next part we will build a logistic regression model for the data classification.

## Part 3: Prepare data and build the logistic regression model (7 points)


**Question 3.1 (2 points)** Apply the "train_test_split" function in the *sklearn* package to split the data in 70% for training and 30% for testing.  Using common variable names like x_train, y_train, x_test and y_test might help later.

In [ ]:
train_vectors, test_vectors, train_labels, test_labels= train_test_split(yeast2.drop('class', axis=1),yeast2['class'], test_size=0.3)
#train_test_split(x,y,test_size=.25) vectors are x, labels are y
#ensure you .drop whatever youre using for y, from x so its not "perfect"

**Question 3.2 (2 points)** Perform the logistic regression. 
* Discuss your results. How well does your model fit your data? What evidence are you using to make the determination? 
* Based on the P values under "P > |z|", which two features **in this dataset** are the least significant and can be dropped?

In [ ]:
#log regression
#from day09in
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors)) #labels are y, vectors are x
result = logit_model.fit()
print(result.summary() )

#the model does not fit that well with the data. the pseurdo rsqu is quite low
#gvh and mit are the most significant
#.alm, erl, nuc and mcg are the least significant

**Question 3.3 (3 points)** Drop the two least important features found in the previous question and perform the logistic regression again. Then use the use the `sklearn.metrics` we imported at the top and run the `accuracy_score` on the 0/1 predicted label and the test labels, and print the accuracy of this model.

* Discuss your results. How well does your reduced model fit your data? What evidence are you using to make the determination?

In [ ]:
#drop 2 worst performers
reduced_yeast=yeast2.drop(['erl','nuc'], axis=1)
#resort
train_vectors, test_vectors, train_labels, test_labels= train_test_split(reduced_yeast.drop('class', axis=1),reduced_yeast['class'], test_size=0.3)

#remodel
logit_model2 = sm.Logit(train_labels, sm.add_constant(train_vectors)) #labels are y, vectors are x
result2 = logit_model2.fit()
print(result2.summary() )
print(metrics.accuracy_score(train_labels, result2.predict())) #ytrue, #ypredicted, 
#My model is predicting non 0/1 values and the accuracy score doesnt like that, I am not sure how to fix

# K-Nearest Neighbors

In the next part we will be building a class that will use the k-nearest neighbors algorithm (kNN) to make predictions on the same dataset. From the previous part (logistic regression), you have selected **4 features** that are important for classification. We will **only** use those 4 features in this part.


## Part 4: KNN classifier, cross-validation and hyperparameter tuning (10 points)

**Question 4.1 (3 points)** Test drive the KNN classifier. Use the same train and test data you created in question 3.4 to build a KNN classifier with K=3. 
- make a `KNeighborsClassifier` with an argument of `n_neighbors=3`. This returns a knn classifier (let's just call it `knn`)
- call `knn.fit` on the training data
- use `knn.predict` on the testing data to generate the predicted values.
- print the confusion matrix.
- print the train and test score using `knn.score`.
- plot the ROC curve with the diagonal (the "chance line") also labeled. Using `sklearn.metrics`, print the `auc` for this model.

In [ ]:
#make a KNeighborsClassifier 
knn=KNeighborsClassifier(n_neighbors=3)

#use knn.fit on training data
knn.fit(train_vectors, train_labels)# knn.fit(x,y) 
#QUESTION so does this just work in place?

#use knn.predict on testing data to generate predicted values (aka labels)
predictedknn=knn.predict(test_vectors) #.predict(x) returns y (predicted labels)

#print confusion matrix
#prints 2x2 matrix with columns of actual[0,1] rows of predicted [0,1] (I think)
print(confusion_matrix(test_labels, predictedknn, labels = [0,1])) #..matrix(y_true,y_predict,labels)

#print train and test score using knn.score
print('training score is',knn.score(train_vectors,train_labels)) #.score(x=test samples,y=true labels for x) 
print('test score is',knn.score(test_vectors,test_labels)) #.score(x=test samples,y=true labels for x) 


In [ ]:
#plot ROC curve w/ diagonal (choice line ) also labeled  ALSO using sklearn.metrics print the auc for this model
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_roc_curve.html#sklearn.metrics.plot_roc_curve
fpr,tpr,thresholds=metrics.roc_curve(test_labels, predictedknn)
roc_auc=metrics.auc(fpr,tpr) #area under the curve
#plt.title('ROC curve')
plt.plot(fpr,tpr,'b',label ='AUC = %0.2f' %roc_auc)

plt.plot([0,1],[0,1],'r--',label='Chance Line') #chance line labeled in legend
plt.legend(loc='lower right')

plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC curve of kNN')
plt.show()

## k-Fold Cross-Validation
Cross-validation is when the dataset is randomly split up into ‘k’ groups. One of the groups is used as the test set and the rest are used as the training set. The model is trained on the training set and scored on the test set. Then the process is repeated until each unique group as been used as the test set.
For example, for 5-fold cross validation, the dataset would be split into 5 groups, and the model would be trained and tested 5 separate times so each group would get a chance to be the test set. This can be seen in the graph below.

<img src="https://miro.medium.com/max/1400/1*NyvaFiG_jXcGgOaouumYJQ.jpeg" width=700px>

The train-test-split method we used in earlier is called ‘holdout’. Cross-validation is better than using the holdout method because the holdout method score is dependent on how the data is split into train and test sets. Cross-validation gives the model an opportunity to test on multiple splits so we can get a better idea on how the model will perform on unseen data.




**Question 4.2 (2 points)** Look up `cross_val_score` in `sklearn.model_selection`. We will still use n_neighbors=3, and  a cross-validation value of 5. `cross_val_score` takes in our k-NN model and our data as parameters. Then it splits our data into 5 groups and fits and scores our data 5 seperate times, recording the accuracy score in an array each time. We will save the accuracy scores in the cv_scores variable. Then find the average of the cv_scores, that will provide you a more accurate understanding of the accuracy of the model.

* Discuss your results. How well do your models fit your data? 
* What are you using to judge that fit (i.e., how should we think about the accuracy score as a measure of quality of the model)?
* How does the quality of the KNN model compare to logistic regression?

In [ ]:
### Put your code here ###
cv_scores=cross_val_score(knn,reduced_yeast.drop('class', axis=1),yeast2['class'])
print('the average of the scores is', np.average(cv_scores))

#answers to questions
#A: the results seem to match with my data, the auc is 0.73 which is close to the 0.769 suggested by the cross val score

#A: Im comparing the average cv score to the auc from the knn model

#A: using the psuedo r-squ value as an approximation of accuracy for the log model, the knn model is much better at 0.769 compared to 0.31 in the log model 

## Hyperparameter tuning


Almost all machine learning models have hyperparamters. Hyperparameters are setting(s) in the model that the user needs to choose before learning takes place. For example, in k-nearest neighbors, the number of neighbors to consider  n_neighbors, is the hyperparameter. An important task in machine learning is hyperparameter tuning, which is finding the optimal hyperparmeter. We will now explore the optimal choice of this parameter for this dataset.

**Question 4.3 (3 points)** Consider the range of `n_neighbors` from 1 to 100, and fix the cross-validation value to be 5. 
- For each value of n_neighbors, compute the means of the cv_scores. 
- Make a plot with the x-axis being n_neighbors, y-axis being the mean of cv_scores.
- Find the optimal choice of n_neighbors with the largest value of the mean of cv_scores.

Discuss your results
* How does the quality of this model compare to the earlier models that you made with KNN and logisitic regression?


In [ ]:
train_vectors2, test_vectors2, train_labels2, test_labels2= train_test_split(reduced_yeast.drop('class', axis=1),reduced_yeast['class'], test_size=0.3)


In [ ]:
#I shall now do this test 100 times 


#initiallize list
score_list=[]
neighbors_list=[]
double_list=[]
#for range n_neighbors from 1 to 100
#make a KNeighborsClassifier 
for i in range(1,101):
    aknn=KNeighborsClassifier(n_neighbors=i)

    #use knn.fit on training data
    aknn.fit(train_vectors2, train_labels2)# knn.fit(x,y) 
    #QUESTION so does this just work in place?

    #use knn.predict on testing data to generate predicted values (aka labels)
    predictedknn2=aknn.predict(test_vectors2) #.predict(x) returns y (predicted labels)
    #generate cv score and add to list
    cv_score=cross_val_score(aknn,reduced_yeast.drop('class', axis=1),reduced_yeast['class'])
    score_list.append([np.average(cv_score)])
    neighbors_list.append(i)
    double_list.append([str(i),[np.average(cv_score)]])
    #returns score_list, [average score] and neighbors_list[#neighbors]


In [ ]:

#x neighbors, y mean of cv scores
plt.plot(neighbors_list,score_list)
print(max(score_list))

#the best value is at 13 neighbors and is 0.8245

Now we will use a more efficient method: `GridSearchCV` in `sklearn.model_selection` to find the optimal n_neighbors.

**Question 4.4 (2 points)** Look up `GridSearchCV` in `sklearn.model_selection`. We will still use a cross-validation value of 5.  Use `best_params_` in `GridSearchCV` to find the optimal n_neighbors. Does it agree with the results from question 4.3?

In [ ]:
#turn the list into a dict, because this premadona of a function requires it
double_dict=dict(double_list)
print(double_dict)

In [ ]:

clf=GridSearchCV(aknn,double_dict,cv=5, return_train_score=True) #param grid are values you want to test and compare aka hyperparamets we are trying to optimize
#aka n_neighbors
#OK SO THE PARAM GRID ITEMS NEEDS ENTRIES TO BE A LIST, EVEN A LIST OF 1. KCOOL. 
clf.fit(train_vectors2,train_labels2)
#OK SO PARAMS MUST BE STR. KCOOL.

#I have no idea if this agrees or disagrees with the other fcn, I have tried changing things to str and reading the documentation for the GridSearchCV fcn but I have NO idea what it is asking for


---
### Assignment wrap-up

Please fill out the form that appears when you run the code below.  **You must completely fill this out in order to receive credit for the assignment!**

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://docs.google.com/forms/d/e/1FAIpQLSc0IBD2mdn4TcRyi-KNXVtS3aEg6U4mOFq2MOciLQyEP4bg1w/viewform?usp=sf_link" 
	width="800px" 
	height="600px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

### Congratulations, you're done!

Submit this assignment by uploading it to the course Desire2Learn web page.  Go to the "Homework Assignments" folder, find the dropbox link for Homework 3, and upload your notebook **and the script you wrote**.